## Passo 1 carregar base de fraudes de credito

In [16]:
import pandas as pd
import numpy as np
from sklearn import svm

#!wget -O MLs15c_creditcard.csv "https://upfluxpm-my.sharepoint.com/:x:/g/personal/cleitonsg_upflux_net/Ef7WwCV3tRpFgPwCfpUrSYQBsUZc3F7uqp5dYcvZj2sdgw?e=xkgflu&download=1"
cc = pd.read_csv("MLs15c creditcard.csv")
cc.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Passo 2 Preparar base de treinamento e teste

In [17]:
#Vamos alterar o nome de class para category
cc= cc.rename(columns={'Class': 'Category'})
# Vamos dividir o dataframe cc baseado nos labels.
nor_obs = cc.loc[cc.Category==0]    # noramal
ano_obs = cc.loc[cc.Category==1]    # anomalias

In [18]:
# Criando base de treinamento com as primeiras 200 mil instâncias normais
train_feature = nor_obs.loc[0:100000, :]
train_feature = train_feature.drop('Category', axis=1)

In [19]:
X_test_1 = nor_obs.loc[200000:, :].drop('Category',axis=1) # Criando base de testes com as normais
X_test_2 = ano_obs.drop('Category',axis=1)
X_test = pd.concat([X_test_1, X_test_2], ignore_index=True)

# O conjunto de dados restante é (após 200.000 observações) anexado com observações anômalas
Y_1 = nor_obs.loc[200000:, 'Category']
Y_2 = ano_obs['Category']

Y_test = pd.concat([Y_1, Y_2], ignore_index=True)
#Y_test é usado para avaliação do modelo used to evaluste the model

# Passo 3 Treinar

In [20]:
# Configurar hyperparameters do SVM One Class
oneclass = svm.OneClassSVM(kernel='linear', gamma=0.001, nu=0.95)
# Muitas opcoes de hyperparameters como: linear, rbf, poly / gamma: 0.001, 0.0001 / nu: 0.25, 0.5, 0.75, 0.95
oneclass.fit(train_feature)

c:\Python310\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


OneClassSVM(gamma=0.001, kernel='linear', nu=0.95)

## Passo 4 Avaliar

In [21]:
fraud_pred = oneclass.predict(X_test)
# Verifique o número de outliers preditos pelo modelo
unique, counts = np.unique(fraud_pred, return_counts=True)
print(np.asarray((unique, counts)).T)

c:\Python310\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


[[   -1   217]
 [    1 84975]]


In [22]:
# Converter Y-test e fraud_pred em um dataframe
Y_test = Y_test.to_frame()
Y_test = Y_test.reset_index()
fraud_pred = pd.DataFrame(fraud_pred)
fraud_pred = fraud_pred.rename(columns={0: 'prediction'})

In [23]:
TP = FN = FP = TN = 0
for j in range(len(Y_test)):
    if Y_test['Category'][j]== 0 and fraud_pred['prediction'][j] == 1:
        TP = TP+1
    elif Y_test['Category'][j]== 0 and fraud_pred['prediction'][j] == -1:
        FN = FN+1
    elif Y_test['Category'][j]== 1 and fraud_pred['prediction'][j] == 1:
        FP = FP+1
    else:
        TN = TN+1
print (TP,  FN,  FP,  TN)

84700 0 275 217


In [24]:
accuracy = (TP+TN)/(TP+FN+FP+TN)
print (accuracy)

0.9967719973706451


## Uma vez que a classe SVM mostrou um desempenho muito promissor para este conjunto de dados com mais de 99% de detecção de anomalia e muito poucos alarmes falsos.